# NLP Assignment 2
# Harman Singh 2019042
# Yash Bhargava 2019289

In [1]:
#Importing essential libraries
import numpy as np
import re
import pandas as pd
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from tqdm import tqdm
import copy
import random
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import TweetTokenizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download("vader_lexicon")
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
#Preprocessing

#Converting to lowercase
def to_lower_case(text):
  text = text.lower()
  return text

#Substituting words with apostrophe with their possible base form
def substitute_text(text):
  text_list = []
  substitutes = {"aren't": 'are not', "can't": 'cannot', "couldn't": 'could not', "didn't": 'did not', "doesn't": 'does not', "don't": 'do not', "hadn't": 'had not', "hasn't": 'has not', "haven't": 'have not', "he'd": 'he would', "he'll": 'he will', "he's": 'he is', "i'd": 'i would', "i'll": 'i will', "i'm": 'i am', "isn't": 'is not', "it's": 'it is', "it'll": 'it will', "i've": 'i have', "let's": 'let us', "mightn't": 'might not', "mustn't": 'must not',"n't": 'not', "shan't": 'shall not', "she'd": 'she would', "she'll": 'she will', "she's": 'she is', "shouldn't": 'should not', "that's": 'that is', "there's": 'there is', "they'd": 'they would', "they'll": 'they will', "they're": 'they are', "they've": 'they have', "we'd": 'we would', "we're": 'we are', "weren't": 'were not', "we've": 'we have', "what'll": 'what will', "what're": 'what are', "what's": 'what is', "what've": 'what have', "where's": 'where is', "who'd": 'who would', "who'll": 'who will', "who're": 'who are', "who's": 'who is', "who've": 'who have', "won't": 'will not', "wouldn't": 'would not', "you'd": 'you would', "you'll": 'you will', "you're": 'you are', "you've": 'you have', "'re": ' are', "wasn't": 'was not', "we'll": 'we will', "'cause": 'because', "could've": 'could have', "how'd": 'how did', "how'd'y": 'how do you', "how'll": 'how will', "how's": 'how is', "I'd": 'I would', "I'd've": 'I would have', "I'll": 'I will', "I'll've": 'I will have', "I'm": 'I am', "I've": 'I have', "i'd've": 'i would have', "i'll've": 'i will have', "it'd": 'it would', "it'd've": 'it would have', "it'll've": 'it will have', "ma'am": 'madam', "mayn't": 'may not', "might've": 'might have', "mightn't've": 'might not have', "must've": 'must have', "mustn't've": 'must not have', "needn't": 'need not', "needn't've": 'need not have', "o'clock": 'of the clock', "oughtn't": 'ought not', "oughtn't've": 'ought not have', "sha'n't": 'shall not', "shan't've": 'shall not have', "she'd've": 'she would have', "she'll've": 'she will have', "should've": 'should have', "shouldn't've": 'should not have', "so've": 'so have', "so's": 'so as', "this's": 'this is', "that'd": 'that would', "that'd've": 'that would have', "there'd": 'there would', "there'd've": 'there would have', "here's": 'here is', "they'd've": 'they would have', "they'll've": 'they will have', "to've": 'to have', "we'd've": 'we would have', "we'll've": 'we will have', "what'll've": 'what will have', "when's": 'when is', "when've": 'when have', "where'd": 'where did', "where've": 'where have', "who'll've": 'who will have', "why's": 'why is', "why've": 'why have', "will've": 'will have', "won't've": 'will not have', "would've": 'would have', "wouldn't've": 'would not have', "y'all": 'you all', "y'all'd": 'you all would', "y'all'd've": 'you all would have', "y'all're": 'you all are', "y'all've": 'you all have', "you'd've": 'you would have', "you'll've": 'you will have'}
  text_tokenised = text.split()
  for i in text_tokenised:
    if(i in substitutes):
      text_list.append(substitutes[i])
    else:
      text_list.append(i)
  return ' '.join(text_list)


#Removing punctuations
def remove_punctuations(text):
  punctuations = '''!()-[|]`{};:'"\,<>./?@#$=+%^&*_~'''
  new_list = ['' if x in punctuations else x for x in text]
  new_list_final = []
  for token in new_list:
    new_token=""
    for char in token:
      if(char not in punctuations):
        new_token+=char
    if(len(new_token)!=0):
      new_list_final.append(new_token)
  return new_list_final

#Removing stopwords
def remove_stopwords(text):
  stopword = stopwords.words('english')
  new_list = [x for x in text if x not in stopword]
  return new_list
  
#Stemming
def stemming(text):
  stemmer = PorterStemmer()
  new_list = [stemmer.stem(x) for x in text]
  return new_list

#Tokenization
def tokenization(text):
  tk = TweetTokenizer()
  return tk.tokenize(text)

def perform_url_html_tag_removal(text):
  new_text = []
  for tt in text:
    temp = re.sub(r'^(https?|www\.|tinyurl|t\.co).*|.*\.(com|org|edu).*|^<.*>$', '', tt)
    temp2 = re.sub(r'^(@|#).*', '', temp)
    if temp2!='':
      new_text.append(temp2)
  return new_text

#Combining all preprocessing steps together for a text
def text_preprocess(text):
  text = to_lower_case(text)
  text = substitute_text(text)
  text = tokenization(text)
  #text = remove_stopwords(text)
  text = remove_punctuations(text)
  text = perform_url_html_tag_removal(text)
  text = [x.strip() for x in text if len(x.strip())]
  #text = stemming(text)
  return text

def data_preprocess(df, col_name):
  data = df.copy(deep=True)
  data[col_name] = data[col_name].apply(text_preprocess)
  return data

In [3]:
#Loading dataset

def load_dataset(path):
  '''Loads the dataset (in CSV format) from the path provided

  Parameters:
  ------------
  path: str
     OS path at which the file is located
    
  Returns
  ------------
  pandas.core.frame.DataFrame
     Pandas dataframe object consisting of the data

  '''

  df = pd.read_csv(path)
  return df

In [4]:
if __name__ == "__main__":
  #Loading dataset
  df = load_dataset(r"E:\Sem7\NLP\Assignments\Assignment 2\A2_dataset.csv")
  test_df = load_dataset(r"E:\Sem7\NLP\Assignments\Assignment 2\A2_test_dataset.csv")

In [5]:
#Applying preprocessing steps
df_preprocessed = data_preprocess(df, 'TEXT')

In [6]:
df.head()

,LABEL,DATE_TIME,TEXT
0,0,Fri Jun 05 14:26:50 2009,About to get threaded and scared
1,1,Thu May 14 10:13:55 2009,@awaisnaseer I like Shezan Mangooo too!!! I ha...
2,1,Fri Jun 05 21:02:20 2009,worked on my car after work. showering then go...
3,1,Sun Jun 14 22:25:52 2009,@Marama Actually we start this afternoon! I w...
4,1,Sun May 31 00:42:12 2009,@gfalcone601 Aww Gi.don't worry.we'll vote for...


In [7]:
df_preprocessed.head()

,LABEL,DATE_TIME,TEXT
0,0,Fri Jun 05 14:26:50 2009,"[about, to, get, threaded, and, scared]"
1,1,Thu May 14 10:13:55 2009,"[awaisnaseer, i, like, shezan, mangooo, too, i..."
2,1,Fri Jun 05 21:02:20 2009,"[worked, on, my, car, after, work, showering, ..."
3,1,Sun Jun 14 22:25:52 2009,"[marama, actually, we, start, this, afternoon,..."
4,1,Sun May 31 00:42:12 2009,"[gfalcone601, aww, gidon, t, worrywe, ll, vote..."


In [8]:
#Pickling preprocessed dataframe
pickle_out = open(r"E:\Sem7\NLP\Assignments\Assignment 2\A2_df_preprocessed.pickle","wb")
pickle.dump(df_preprocessed, pickle_out)
pickle_out.close()

In [9]:
df_preprocessed = pickle.load(open(r"E:\Sem7\NLP\Assignments\Assignment 2\A2_df_preprocessed.pickle","rb"))

In [10]:
#Getting vocabulary of our data
def get_vocab(data, col_name):
  vocab = set()
  for text in data[col_name]:
    vocab.update(text)
  return vocab


In [11]:
#Constructing co-occurrence matrix
def construct_bigram_model(data, col_name, vocab):

  bigrams = []
  co_occurrence_matrix = {}
  unigram_count = {}



  #Initialising bigram counts with 0
  for token in tqdm(vocab):
    co_occurrence_matrix[token] = {}
    unigram_count[token] = 0
    for token_n in vocab:
      co_occurrence_matrix[token][token_n] = 0


  
  #Updating bigram and unigram counts
  for tokenised_text in tqdm(list(data[col_name])):

    unigram_count[tokenised_text[0]]+=1

    for i in range(1, len(tokenised_text)):

      #increasing unigram count
      unigram_count[tokenised_text[i]]+=1

      #Updating bigrams
      if((tokenised_text[i-1], tokenised_text[i]) not in bigrams):
        bigrams.append((tokenised_text[i-1], tokenised_text[i]))

      #Increasing bigram count
      co_occurrence_matrix[tokenised_text[i-1]][tokenised_text[i]]+=1
    

  return unigram_count, bigrams, co_occurrence_matrix


In [12]:
#Applying laplace smoothing and returning bigram probabilities
def laplace_smoothed_bigram_model(unigram_count, bigrams, co_occurrence_matrix, vocab_size):

  smoothed_co_occurrence_matrix = {}

  for token in tqdm(co_occurrence_matrix.keys()):
    smoothed_co_occurrence_matrix[token]={}
    for token_n in co_occurrence_matrix[token].keys():
      smoothed_co_occurrence_matrix[token][token_n] = ((co_occurrence_matrix[token][token_n]+1)/(unigram_count[token]+vocab_size))

  return smoothed_co_occurrence_matrix

In [13]:
#Getting vocabulary
vocab = get_vocab(df_preprocessed, 'TEXT')

In [14]:
#Creating bigram model
unigram_count, bigrams, co_occurrence_matrix = construct_bigram_model(df_preprocessed, 'TEXT', vocab)

100%|██████████████████████████████████████████████████████████████████████████████| 4287/4287 [00:56<00:00, 75.57it/s]


In [15]:
#Applying laplace smoothing
smoothed_co_occurrence_matrix = laplace_smoothed_bigram_model(unigram_count, bigrams, co_occurrence_matrix ,len(vocab))

100%|████████████████████████████████████████████████████████████████████████████| 10142/10142 [02:00<00:00, 84.17it/s]


In [16]:
for k in smoothed_co_occurrence_matrix.keys():
  print(smoothed_co_occurrence_matrix[k])
  break

{'twistedtamsyn': 9.859016070196194e-05, 'alba': 9.859016070196194e-05, 'rkuang': 9.859016070196194e-05, 'lime': 9.859016070196194e-05, 'fbb420': 9.859016070196194e-05, 'hungry': 9.859016070196194e-05, 'not': 9.859016070196194e-05, 'honestly': 9.859016070196194e-05, 'hmmx': 9.859016070196194e-05, 'leading': 9.859016070196194e-05, 'development01': 9.859016070196194e-05, 'pc': 9.859016070196194e-05, 'torehtard': 9.859016070196194e-05, 'bundle': 9.859016070196194e-05, 'vomiting': 9.859016070196194e-05, 'arcticpenguin': 9.859016070196194e-05, 'georgiielu': 9.859016070196194e-05, 'replaced': 9.859016070196194e-05, 'sunshinerocker': 9.859016070196194e-05, 'savorthethyme': 9.859016070196194e-05, 'black': 9.859016070196194e-05, 'spundae': 9.859016070196194e-05, 'shanika': 9.859016070196194e-05, 'greenwich': 9.859016070196194e-05, 'favorites': 9.859016070196194e-05, 'everythinggg': 9.859016070196194e-05, 'ross': 9.859016070196194e-05, 'whedonverse': 9.859016070196194e-05, 'nyc': 9.8590160701961

# With Beta

In [ ]:
def calculate_vader_scores(co_occurrence_matrix):
  sent_analyzer = SentimentIntensityAnalyzer()
  vader_dict = {}

  for token in tqdm(co_occurrence_matrix.keys()):
    vader_dict[token] = {}
    for token_n in co_occurrence_matrix[token].keys():
      score = sent_analyzer.polarity_scores(token + " " + token_n)
      vader_dict[token][token_n] = (score['pos'], score['neg'], score['compound'])
  
  return vader_dict

In [ ]:
vader_dict = calculate_vader_scores(co_occurrence_matrix)

In [ ]:
for k in vader_dict.keys():
  print(vader_dict[k])
  break

In [ ]:
random.seed(100)

In [ ]:
#Applying laplace smoothing and returning bigram probabilities
def laplace_smoothed_bigram_model_with_beta_pos(unigram_count, bigrams, co_occurrence_matrix, vader_dict, vocab_size):

  smoothed_co_occurrence_matrix = {}

  sent_analyzer = SentimentIntensityAnalyzer()
  # predicted_scores = [sent_analyzer.polarity_scores(text)['compound'] for text in data]

  for token in tqdm(co_occurrence_matrix.keys()):
    smoothed_co_occurrence_matrix[token] = {}
    for token_n in co_occurrence_matrix[token].keys():
      smoothed_co_occurrence_matrix[token][token_n] = ((co_occurrence_matrix[token][token_n]+1)/(unigram_count[token]+vocab_size)) + 0.0001*vader_dict[token][token_n][0]

  return smoothed_co_occurrence_matrix

In [ ]:
#Applying laplace smoothing and returning bigram probabilities
def laplace_smoothed_bigram_model_with_beta_neg(unigram_count, bigrams, co_occurrence_matrix, vader_dict, vocab_size):

  smoothed_co_occurrence_matrix = {}

  sent_analyzer = SentimentIntensityAnalyzer()
  # predicted_scores = [sent_analyzer.polarity_scores(text)['compound'] for text in data]

  for token in tqdm(co_occurrence_matrix.keys()):
    smoothed_co_occurrence_matrix[token] = {}
    for token_n in co_occurrence_matrix[token].keys():
      smoothed_co_occurrence_matrix[token][token_n] = ((co_occurrence_matrix[token][token_n]+1)/(unigram_count[token]+vocab_size)) + 0.0001*vader_dict[token][token_n][1]

  return smoothed_co_occurrence_matrix

In [ ]:
#Applying laplace smoothing and returning bigram probabilities
def laplace_smoothed_bigram_model_with_beta_compound(unigram_count, bigrams, co_occurrence_matrix, vader_dict, vocab_size):

  smoothed_co_occurrence_matrix = {}

  sent_analyzer = SentimentIntensityAnalyzer()
  # predicted_scores = [sent_analyzer.polarity_scores(text)['compound'] for text in data]

  for token in tqdm(co_occurrence_matrix.keys()):
    smoothed_co_occurrence_matrix[token] = {}
    for token_n in co_occurrence_matrix[token].keys():
      smoothed_co_occurrence_matrix[token][token_n] = ((co_occurrence_matrix[token][token_n]+1)/(unigram_count[token]+vocab_size)) + 0.0001*abs(vader_dict[token][token_n][2])

  return smoothed_co_occurrence_matrix

In [ ]:
#Applying laplace smoothing with beta positive
smoothed_co_occurrence_matrix_with_beta_pos = laplace_smoothed_bigram_model_with_beta_pos(unigram_count, bigrams, co_occurrence_matrix, vader_dict ,len(vocab))

In [ ]:
#Applying laplace smoothing with beta negative
smoothed_co_occurrence_matrix_with_beta_neg = laplace_smoothed_bigram_model_with_beta_neg(unigram_count, bigrams, co_occurrence_matrix, vader_dict ,len(vocab))

# Helper Functions

In [17]:
def calculate_sentence_perplexity(LM, unigram_dict, vocab_size, sentence):
  '''
  Parameters:
    LM (Language Model): dict of dict
    unigram_dict: dict
    vocab_size: int
    sentence: string
  
  Returns:
    Perplexity of a sentence
  
  Note: Probabilty is calculated in log space to prevent overflow/underflow in calculations
  '''

  tokens_list = sentence.split()
  N = len(tokens_list)
  probability = 1

  probability *= unigram_dict[tokens_list[0]]/vocab_size

  for i in range(1,len(tokens_list)):
    probability *= LM[tokens_list[i-1]][tokens_list[i]]
  
  perplexity = np.power( 1/probability, (1/N) )

  return perplexity

In [18]:
def calculate_sentence_perplexity_in_log_space(LM, unigram_dict, vocab_size, sentence):
  '''
  Parameters:
    LM (Language Model): dict of dict
    unigram_dict: dict
    vocab_size: int
    sentence: string
  
  Returns:
    Perplexity of a sentence
  
  Note: Probabilty is calculated in log space to prevent overflow/underflow in calculations
  '''

  tokens_list = sentence.split()
  N = len(tokens_list)
  log_probability = 0

  log_probability += np.log(unigram_dict[tokens_list[0]]/vocab_size)

  for i in range(1,len(tokens_list)):
    log_probability += np.log(LM[tokens_list[i-1]][tokens_list[i]])
  
  perplexity = np.power( -1/log_probability, (1/N) )

  return perplexity

In [19]:
def calculate_average_perplexity(LM, unigram_dict, vocab_size, sentences_list):
  '''
  Parameters:
    LM (Language Model): dict of dict
    unigram_dict: dict
    vocab_size: int
    sentences_list: list of strings
  
  Returns: None
  '''

  total_perplexity = 0
  total_perplexity_in_log_space = 0
  num_of_sentences = len(sentences_list)

  for sentence in sentences_list:
    sent_perplexity = calculate_sentence_perplexity(LM, unigram_dict, vocab_size, sentence)
    total_perplexity += sent_perplexity
    sent_perplexity_in_log_space = calculate_sentence_perplexity_in_log_space(LM, unigram_dict, vocab_size, sentence)
    total_perplexity_in_log_space += sent_perplexity_in_log_space
  
  avg_perplexity = total_perplexity/num_of_sentences
  avg_perplexity_in_log_space = total_perplexity_in_log_space/num_of_sentences

  print('Average Perplexity: {}'.format( avg_perplexity))
  print('Average Perplexity In Log Space: {}'.format( avg_perplexity_in_log_space))

In [20]:
def train_and_test_model(train_dataset_param, test_dataset_param):
  '''
  Parameters:
    train_dataset: pandas.core.frame.DataFrame
    test_dataset: pandas.core.frame.DataFrame
  
  Returns: float

  Trains a ML Model on the train_data and train_labels and return accuracy calculated using test_data and test_labels
  '''

  train_dataset = train_dataset_param.copy(deep=True)
  test_dataset = test_dataset_param.copy(deep=True)

  train_dataset['TRAIN_DATA'] = [' '.join(x) for x in train_dataset['TEXT']]
  test_dataset['TRAIN_DATA'] = [' '.join(x) for x in test_dataset['TEXT']]


  # Randomly shuffle dataset
  train_dataset_shuffled = shuffle(train_dataset, random_state=0)

  # train data
  train_sentences = train_dataset_shuffled['TRAIN_DATA']
  train_labels = train_dataset_shuffled['LABEL']

  # test data
  test_sentences = test_dataset['TRAIN_DATA']
  test_labels = test_dataset['LABEL']

  # model pipeline
  model = make_pipeline(TfidfVectorizer(), MultinomialNB())
  
  # training model on train data
  model.fit(train_sentences, train_labels)

  # predicting on test data
  predicted_test_labels = model.predict(test_sentences)

  # accuracy on test data
  test_accuracy = accuracy_score(test_labels, predicted_test_labels)

  return test_accuracy

In [21]:
def extrinsic_evaluation(dataset_A, dataset_B, dataset_test):
  '''
  Parameters:
    dataset_A: pandas.core.frame.DataFrame
    dataset_B: pandas.core.frame.DataFrame
    dataset_test: pandas.core.frame.DataFrame
  
  Returns: None

  Trains ML model on dataset_A & dataset_B separately and reports their accuracy on test set
  '''

  accuracy_A = train_and_test_model(dataset_A, dataset_test)
  accuracy_B = train_and_test_model(dataset_B, dataset_test)

  print('Accuracy on test set after training on dataset A: {}'.format(accuracy_B))
  print('Accuracy on test set after training on dataset B: {}'.format(accuracy_A))

# Part A a)
Storing bigram model after smoothing

In [ ]:
#Pickling co_occurrence_matrix
pickle_out = open("/content/drive/MyDrive/Sem-7_NLP/A2_co_occurrence_matrix.pickle","wb")
pickle.dump(co_occurrence_matrix, pickle_out)
pickle_out.close()

#Pickling unigram count
pickle_out = open("/content/drive/MyDrive/Sem-7_NLP/A2_unigram_count.pickle","wb")
pickle.dump(unigram_count, pickle_out)
pickle_out.close()

#Pickling smoothed co_occurrence_matrix
pickle_out = open("/content/drive/MyDrive/Sem-7_NLP/A2_smoothed_co_occurrence_matrix.pickle","wb")
pickle.dump(smoothed_co_occurrence_matrix, pickle_out)
pickle_out.close()

In [ ]:
#Pickling bigrams
pickle_out = open("/content/drive/MyDrive/Sem-7_NLP/A2_bigrams.pickle","wb")
pickle.dump(bigrams, pickle_out)
pickle_out.close()

In [ ]:
co_occurrence_matrix = pickle.load(open("/content/drive/MyDrive/Sem-7_NLP/A2_co_occurrence_matrix.pickle","rb"))
unigram_count = pickle.load(open("/content/drive/MyDrive/Sem-7_NLP/A2_unigram_count.pickle","rb"))
smoothed_co_occurrence_matrix = pickle.load(open("/content/drive/MyDrive/Sem-7_NLP/A2_smoothed_co_occurrence_matrix.pickle","rb"))
bigrams = pickle.load(open("/content/drive/MyDrive/Sem-7_NLP/A2_bigrams.pickle","rb"))

In [22]:
import bz2
import _pickle as cPickle

In [23]:
def decompress_pickle(file):
    data = bz2.BZ2File(file,'rb')
    data = cPickle.load(data)
    return data

In [24]:
vd_pos = decompress_pickle(r"E:\Sem7\NLP\Assignments\Assignment 2\A2_vader_dict_pos.pbz2")

In [27]:
del vd_pos

In [28]:
vd_neg = decompress_pickle(r"E:\Sem7\NLP\Assignments\Assignment 2\A2_vader_dict_neg.pbz2")

In [ ]:
vd_compound = decompress_pickle(r"E:\Sem7\NLP\Assignments\Assignment 2\A2_vader_dict_compound.pbz2")

In [ ]:
#Generating a random sentence
def generate_random_sentence(start_word, bigram_probabilities, sentence_len):
  start = start_word
  next_word_prob = 0
  sentence = []
  sentence.append(start)
  final_next_word = ''
  for i in range(sentence_len-1):
    next_word_prob = 0
    final_next_word = ''
    for next_word in bigram_probabilities[start]:
      if(bigram_probabilities[start][next_word]>next_word_prob):
        next_word_prob = bigram_probabilities[start][next_word]
        final_next_word = next_word
    sentence.append(final_next_word)
    start = final_next_word
  return ' '.join(sentence)

In [ ]:
#Generating a random sentence
def generate_random_sentence_2(start_word, bigram_probabilities, sentence_len, vader_dict, idx):
  start = start_word
  next_word_prob = 0
  sentence = []
  sentence.append(start)
  final_next_word = ''
  for i in range(sentence_len-1):
    next_word_prob = 0
    final_next_word = ''
    for next_word in bigram_probabilities[start]:
      if(bigram_probabilities[start][next_word] + 0.0001*abs(vader_dict[start][next_word][idx]) > next_word_prob):
        next_word_prob = bigram_probabilities[start][next_word]
        final_next_word = next_word
    sentence.append(final_next_word)
    start = final_next_word
  return ' '.join(sentence)

In [ ]:
#Generating a random sentence
def generate_random_sentence_3(start_word, bigram_probabilities, sentence_len, vader_dict, idx, k_param):
    start = start_word
    next_word_prob = 0
    sentence = []
    sentence.append(start)
    final_next_word = ''
    for i in range(sentence_len-1):
        next_word_prob = 0
        final_next_word = ''
        final_next_word_possibilities = []
        for next_word in bigram_probabilities[start]:
            final_next_word_possibilities.append((next_word, bigram_probabilities[start][next_word] + 0.0001*vader_dict[start][next_word][idx]))
        final_next_word_possibilities.sort(key = lambda x: x[1], reverse=True)
        selected_idx = random.randint(0,k_param)
        sentence.append(final_next_word_possibilities[selected_idx][0])
        start = final_next_word_possibilities[selected_idx][0]
    return ' '.join(sentence)

In [ ]:
#Generating a random sentence
def generate_random_sentence_4(start_word, bigram_probabilities, sentence_len, vader_dict):
  start = start_word
  next_word_prob = 0
  sentence = []
  sentence.append(start)
  final_next_word = ''
  for i in range(sentence_len-1):
    next_word_prob = 0
    final_next_word = ''
    for next_word in bigram_probabilities[start]:
      if(bigram_probabilities[start][next_word] + 0.0001*abs(vader_dict[start][next_word]) > next_word_prob):
        next_word_prob = bigram_probabilities[start][next_word]
        final_next_word = next_word
    sentence.append(final_next_word)
    start = final_next_word
  return ' '.join(sentence)

In [ ]:
#Generating a random sentence
def generate_random_sentence_5(start_word, bigram_probabilities, sentence_len, vader_dict, k_param):
    start = start_word
    next_word_prob = 0
    sentence = []
    sentence.append(start)
    final_next_word = ''
    for i in range(sentence_len-1):
        next_word_prob = 0
        final_next_word = ''
        final_next_word_possibilities = []
        for next_word in bigram_probabilities[start]:
            final_next_word_possibilities.append((next_word, bigram_probabilities[start][next_word] + 0.0001*vader_dict[start][next_word]))
        final_next_word_possibilities.sort(key = lambda x: x[1], reverse=True)
        selected_idx = random.randint(0,k_param)
        sentence.append(final_next_word_possibilities[selected_idx][0])
        start = final_next_word_possibilities[selected_idx][0]
    return ' '.join(sentence)

In [33]:
#Generating a random sentence
def generate_random_sentence_6(start_word, bigram_probabilities, sentence_len, vader_dict, k_param):
    start = start_word
    next_word_prob = 0
    sentence = []
    sentence.append(start)
    final_next_word = ''
    for i in range(sentence_len-1):
        next_word_prob = 0
        final_next_word = ''
        final_next_word_possibilities = []
        for next_word in bigram_probabilities[start]:
            final_next_word_possibilities.append((next_word, bigram_probabilities[start][next_word] + 0.0005*vader_dict[start][next_word]))
        final_next_word_possibilities.sort(key = lambda x: x[1], reverse=True)
        selected_idx = random.randint(0,k_param)
        sentence.append(final_next_word_possibilities[selected_idx][0])
        start = final_next_word_possibilities[selected_idx][0]
    return ' '.join(sentence)

Approach 1

In [ ]:
# #Generating sentences
# sentences = []
# sentences_compound = []

# #vocab list
# vocab_list = list(vocab)

# #Generating random numbers
# random_start_words = random.sample(range(len(vocab_list)), 500)

# #Selecting random start words
# for random_idx in range in tqdm(len(random_start_words)):
#     sentences_compound.append(generate_random_sentence_2(vocab_list[random_start_words[random_idx]], smoothed_co_occurrence_matrix, random.randint(8,14), vader_dict, 2))
#     if random_idx % 2 ==0:
#         sentences.append(generate_random_sentence_2(vocab_list[random_start_words[random_idx]], smoothed_co_occurrence_matrix, random.randint(8,14), vader_dict, 0))
#     else:
#         sentences.append(generate_random_sentence_2(vocab_list[random_start_words[random_idx]], smoothed_co_occurrence_matrix, random.randint(8,14), vader_dict, 1))

In [26]:
#Generating sentences
sentences = []
sentences_compound = []
sentences_pos = []

#vocab list
vocab_list = list(vocab)

#Generating random numbers
random_start_words = random.sample(range(len(vocab_list)), 250)

#Selecting random start words
for random_idx in tqdm(range(len(random_start_words))):
    sentences_pos.append(generate_random_sentence_6(vocab_list[random_start_words[random_idx]], smoothed_co_occurrence_matrix, random.randint(8,14), vd_pos, 5))

100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:20<00:00, 12.41it/s]


In [34]:
#Generating sentences
sentences_neg = []

#vocab list
vocab_list = list(vocab)

#Generating random numbers
random_start_words = random.sample(range(len(vocab_list)), 250)

#Selecting random start words
for random_idx in tqdm(range(len(random_start_words))):
    sentences_neg.append(generate_random_sentence_6(vocab_list[random_start_words[random_idx]], smoothed_co_occurrence_matrix, random.randint(8,14), vd_neg, 5))

100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:20<00:00, 12.31it/s]


In [30]:
sentences_pos

['nettz ily honestly honestly honestly excited favorites vip',
 'razzle freedom vip excited favorites vip loved promoting loved favorites honestly promoting excited',
 'ment freedom vip loved excited about funny promoting',
 'kreativeham ily honestly excited about lol jk promoting promoting favorites promoting',
 'petewentz glorious promoting honestly excited favorites vip excited to work',
 'men paradise honestly promoting promoting honestly excited about',
 'lets glorious sunshine promoting favorites excited honestly promoting excited',
 'thts freedom excited about funny promoting vip loved',
 'yer love it would not get it will love it is it',
 'everywhere glorious excited about it would have to see ily promoting favorites excited for',
 'maths love you have to see you too xoxo honestly',
 'â promoting excited honestly honestly love the good night ily promoting excited favorites',
 'defense promoting favorites excited i want to work ily',
 'notes love the last ily honestly loved prom

In [35]:
sentences_neg

['pleaseee killed pressure pressure hates pressure disappointments 7',
 'bathroom tragedy hates me up to go for some more like i was',
 '¢ disappointments disappointments fails pressure disappointments fails fails fails hates hates hates me too',
 'live in my head hell everybody hell everybody raping hates jerk pressure',
 'pissed offi raping pressure fails pressure jerk pressure jerk',
 'cluster kill jerk jerk disappointments 7 disappointments 7 disappointments hates jerk jerk pressure',
 'afairs tragedy jerk disappointments pressure disappointments pressure 7 disappointments jerk 7 disappointments',
 'mission kill me up my friends raping pressure fails',
 'stiff killed jerk disappointments hates me feel so bad i am not',
 '12 hell i have the way kill 7 jerk',
 'bubble hell i do you can you have the day with no',
 'myliverandlungs fucked 7 pressure hates pressure 7 disappointments disappointments hates',
 'limping raping 7 pressure pressure pressure jerk jerk hates disappointments',
 

In [36]:
sentences = sentences_pos + sentences_neg

In [ ]:
sentences_compound

In [37]:
#Performing VADER sentiment analysis

def perform_vader_sentiment_analysis(data):
 
  sent_analyzer = SentimentIntensityAnalyzer()
  predicted_scores = [sent_analyzer.polarity_scores(text)['compound'] for text in data]
  predicted_labels = []
  for score in predicted_scores:
    if(score>=0.05):
      predicted_labels.append(1)
    elif(score<=-0.05):
      predicted_labels.append(0)
    else:
      predicted_labels.append(-1)
  return predicted_labels

Considering neutral labels as positive

In [38]:
#Performing vader sentiment analysis on the generated sentences
y_pred_with_neu = perform_vader_sentiment_analysis(sentences)
y_pred_case1_a = [1 if score==1 or score==-1 else 0 for score in y_pred_with_neu]
df_gen_sentences = pd.DataFrame()
df_gen_sentences['LABEL'] = y_pred_case1_a
df_gen_sentences['DATE_TIME'] = ['NaN' for i in range(len(sentences))]
df_gen_sentences['TEXT'] = sentences



In [39]:
df_gen_sentences.head()

,LABEL,DATE_TIME,TEXT
0,1,NaN,nettz ily honestly honestly honestly excited f...
1,1,NaN,razzle freedom vip excited favorites vip loved...
2,1,NaN,ment freedom vip loved excited about funny pro...
3,1,NaN,kreativeham ily honestly excited about lol jk ...
4,1,NaN,petewentz glorious promoting honestly excited ...


In [40]:
#Computing dataset B (dataset A + 500 generated samples)
data_concat = [df, df_gen_sentences]
df_b = pd.concat(data_concat)

In [41]:
df_b.head()

,LABEL,DATE_TIME,TEXT
0,0,Fri Jun 05 14:26:50 2009,About to get threaded and scared
1,1,Thu May 14 10:13:55 2009,@awaisnaseer I like Shezan Mangooo too!!! I ha...
2,1,Fri Jun 05 21:02:20 2009,worked on my car after work. showering then go...
3,1,Sun Jun 14 22:25:52 2009,@Marama Actually we start this afternoon! I w...
4,1,Sun May 31 00:42:12 2009,@gfalcone601 Aww Gi.don't worry.we'll vote for...


In [43]:
#Pickling Dataset B
pickle_out = open(r"E:\Sem7\NLP\Assignments\Assignment 2\A2_df_b.pickle","wb")
pickle.dump(df_b, pickle_out)
pickle_out.close()

In [ ]:
df_b = pickle.load(open(r"E:\Sem7\NLP\Assignments\Assignment 2\A2_df_b.pickle","rb"))

In [ ]:
# Compound sentences

In [ ]:
#Performing vader sentiment analysis on the generated sentences
y_pred_with_neu_compound = perform_vader_sentiment_analysis(sentences_compound)
y_pred_case1_a_compound = [1 if score==1 or score==-1 else 0 for score in y_pred_with_neu_compound]
df_gen_sentences_compound = pd.DataFrame()
df_gen_sentences_compound['LABEL'] = y_pred_case1_a_compound
df_gen_sentences_compound['DATE_TIME'] = ['NaN' for i in range(len(sentences_compound))]
df_gen_sentences_compound['TEXT'] = sentences_compound

In [ ]:
df_gen_sentences_compound.head()

In [ ]:
#Computing dataset B (dataset A + 500 generated samples)
data_concat_compound = [df, df_gen_sentences_compound]
df_b_compound = pd.concat(data_concat_compound)

In [ ]:
df_b_compound.head()

In [ ]:
#Pickling Dataset B Compound
pickle_out = open(r"E:\Sem7\NLP\Assignments\Assignment 2\A2_df_b_compound.pickle","wb")
pickle.dump(df_b_compound, pickle_out)
pickle_out.close()

# Part A b)
Top 4 bigrams

In [ ]:
#Sorting bigrams
bigram_scores = []
for bigram in bigrams:
  bigram_scores.append((bigram[0], bigram[1], smoothed_co_occurrence_matrix[bigram[0]][bigram[1]]))
bigram_scores.sort(key = lambda x: x[2], reverse=True)

In [ ]:
#Printing top 4 bigrams with scores
for i in range(4):
  print(f"Bigram: {bigram_scores[i][0]} {bigram_scores[i][1]}, Score: {bigram_scores[i][2]}")

In [44]:
# Preprocessing test set
test_df_preprocessed = data_preprocess(test_df, 'TEXT')

In [45]:
# Preprocessing df_b
df_b_preprocessed = data_preprocess(df_b, 'TEXT')

In [ ]:
# Preprocessing df_b
df_b_compound_preprocessed = data_preprocess(df_b_compound, 'TEXT')

# Part A c) & Part B e)

In [46]:
extrinsic_evaluation(df_preprocessed, df_b_preprocessed, test_df_preprocessed)

Accuracy on test set after training on dataset A: 0.9099378881987578
Accuracy on test set after training on dataset B: 0.9177018633540373


In [ ]:
extrinsic_evaluation(df_preprocessed, df_b_compound_preprocessed, test_df_preprocessed)

In [ ]:
def checking():
  df_check = df_preprocessed[:500]
  acc_check = train_and_test_model(df_check, test_df_preprocessed)
  print(acc_check)

  df_500 = df_b_preprocessed[-500:]
  acc_500 = train_and_test_model(df_500, test_df_preprocessed)
  print(acc_500)
checking()

# Part B a)

In [ ]:
# See Report

# Part B b)

In [47]:
df_gen_sentences.to_csv(r"E:\Sem7\NLP\Assignments\Assignment 2\A2_generated_sentences.csv")

In [ ]:
df_gen_sentences_compound.to_csv(r"E:\Sem7\NLP\Assignments\Assignment 2\A2_generated_sentences_compound.csv")

# Part B c)

In [48]:
# Average Perplexity
calculate_average_perplexity(smoothed_co_occurrence_matrix, unigram_count, len(vocab), sentences)

Average Perplexity: 5668.122771775364
Average Perplexity In Log Space: 0.6535713440571285


In [ ]:
# Average Perplexity
calculate_average_perplexity(smoothed_co_occurrence_matrix, unigram_count, len(vocab), sentences_compound)

# Part B d)

In [ ]:
# See Report